In [ ]:
import pandas as pd

In [9]:
df_final_demo = pd.read_csv('df_final_demo.txt')
df_final_experiment_clients = pd.read_csv('df_final_experiment_clients.txt')
df_pt_1 = pd.read_csv('df_pt_1.txt')
df_pt_2 = pd.read_csv('df_pt_2.txt')

In [ ]:
df_final_demo

In [ ]:
df_final_experiment_clients

In [ ]:
df_pt_1

In [ ]:
df_pt_2